<a href="https://colab.research.google.com/github/HieuHongHao/Amazon-review-instruments/blob/main/Amazon_Instrument_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from math import nan
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.python.training.tracking.base import no_manual_dependency_tracking_scope
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences




    

In [ ]:
pwd

'/content'

In [ ]:
#Extract reviews and overall scores
drive.mount('/content/gdrive')
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks')
data = data.dropna() # drop missing values 
data = data[['reviewText','overall']]
reviews = np.array(data['reviewText'])
scores = data['overall']
scores = np.array(scores)
scores = np.array([1 if score >= 3 else 0 for score in scores])


In [ ]:
vocabSize = 10000
reviews = [one_hot(doc,vocabSize,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n') for doc in reviews]

In [ ]:
maxSentLen = 0
for doc in reviews:
  if(len(doc) > maxSentLen):
    maxSentLen = len(doc)
embedded_docs = pad_sequences(reviews,padding="pre",maxlen = maxSentLen)


In [ ]:
embedded_docs

array([[   0,    0,    0, ..., 3235, 2942, 3146],
       [   0,    0,    0, ...,  752, 9556, 3440],
       [   0,    0,    0, ..., 5841, 2683, 9310],
       ...,
       [   0,    0,    0, ..., 2683, 1913,  794],
       [   0,    0,    0, ..., 1526, 1496, 1851],
       [   0,    0,    0, ..., 8766, 8766, 5825]], dtype=int32)

In [ ]:
Vector_features = 100
model = Sequential()
model.add(Embedding(vocabSize,Vector_features,input_length = maxSentLen))
model.add(LSTM(100))  
model.add(Dense(1,activation="sigmoid")) 
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2059, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(np.array(embedded_docs),scores,test_size = 0.33,random_state = 42)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs= 10, batch_size= 64)

In [ ]:
y_pred = model.predict(x_test)
y_pred = (model.predict(x_test) > 0.5).astype("int32")


In [ ]:
from tensorflow.keras.metrics import Accuracy
measures = Accuracy()
measures.update_state(y_test,y_pred)
measures.result().numpy()


0.93244445